In [3]:
import pandas as pd
from os.path import isfile
from tailer import tail
from FinanceDataReader import DataReader
from pykrx import stock
from time import sleep

from pattern_labelling import * 


nsq_p = pd.read_csv("../resources/nasdaq.csv")[['date','nasdaq']]
nsq_p['date'] = pd.to_datetime(nsq_p['date'])

In [6]:
stock_list = pd.read_csv("../resources/stockcode.csv", dtype = {"종목코드": str, "회사명": str})
first_df = pd.read_csv('../resources/ohlcv_p1p2p3_nasdq/000020.csv', parse_dates=['date'], index_col=[0])

pd_last_date = first_df['date'].iloc[-1]
pd_today = pd.to_datetime(pd.Timestamp.now().date())
# 데이터를 갱신할 날짜 범위
pd_last_p1 = pd_last_date + pd.Timedelta(days=1)
pd_drange = pd.date_range(pd_last_p1, pd_today)
if pd_last_p1 == pd_today:
    pd_drange = []

pd_date = pd_today
df_update = stock.get_market_ohlcv_by_ticker(pd_date)

df_update = df_update.reset_index()
stock_ = df_update.iloc[0]
stock_code = stock_['종목코드']
update_target = pd.read_csv(f'../resources/ohlcv_p1p2p3_nasdq/{stock_code}.csv', parse_dates=['date'],
                            index_col=[0])
update_target = update_target.append({'date':pd_date,'open':stock_['시가'],'high':stock_['고가'],'low':stock_['저가'],'close':stock_['종가'],'volume':stock_['거래량']}, 
                            ignore_index=True)
last_idx = len(update_target)-1
update_target['pattern1'].values[last_idx] = labellingD0(update_target.iloc[last_idx])
update_target['pattern2'].values[last_idx] = labellingD1(update_target.iloc[last_idx-1:last_idx+1])
update_target['pattern3'].values[last_idx] = labellingD2(update_target.iloc[last_idx-2:last_idx+1])
try:
    update_target['nasdaq'].values[last_idx] = nsq_p[nsq_p['date']==pd_date].values[0][1]
except IndexError:
    update_target['nasdaq'].values[last_idx] = update_target['nasdaq'].iloc[last_idx-1]


In [8]:
update_target = update_target[:2170]

In [33]:
temp = update_target.values[last_idx-1]
temp[0] = temp[0].date()
for i in range(6):
    temp[i] = str(temp[i])
temp = "".join([temp[i//2] if i%2==0 else "," for i in range(19)])
temp += '\n'
temp

'2020-10-27,16850,18150,15900,17500,851662,P05,M10P05,S07P05,D04\n'

array(['2020-10-27', '16850', '18150', '15900', '17500', '851662', 'P05',
       'M10P05', 'S07P05', 'D04'], dtype=object)